In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Step 0: MACS2 call peaks from HiChIP data
This step is optional if supplied with pre-defined peaks.

To call peaks from religation, dangling end, and dump pairs, we use the mapped and paired bam file as input. The pipeline also provide options for including short cis validpairs determined by `local_range` parameters and whether to exlude interchromosomal validpairs (`exclude_interchro`).

In [2]:
from hichip_object.hichip_callpeaks import call_anchors_from_hichip
import ray, logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s: %(message)s")

Fillin the parameters

In [3]:
digestion_site = "(CT[ATCG]AG)|(TTAA)"  # MiD HiChIP digestion enzymes
genome_fa = "/home/software/mm9.fa"
# for original HiChIP, use merged and paired bam in bwt2/ folder generated by HiC-Pro 
bam_file = "/Extension_HDD2/Hanbin/ES_Cell/E14/HiC3_HL/HL28_Smc1_MiDHiChIP_Test/HL28_Smc1_MiDHiChIP_out/bowtie_results/bwt2_multipass/data/data.mergePasses.bam"
# control the PETs to include. Considering includes more validpairs if sequencing depth is low
local_range = 1000
exclude_interchro = True
# macs2 parameters; relax the stringency if sequencing depth is low
macs2_qval = 0.01
macs2_genome = "mm"
macs2_path = "/home/coco/miniconda3/envs/hichip-loop/bin/macs2"
macs2_out_dir = "/Extension_HDD2/Hanbin/ES_Cell/E14/HiC3_HL/HL28_Smc1_MiDHiChIP_Test/HL28_Smc1_MiDHiChIP_out/"
prefix = "Smc1_HiChIP_1kb"

mapq = 10
nprocs = 30

In [4]:
ray.init(num_cpus=nprocs)

2021-08-04 20:44:02,996	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.0.9',
 'raylet_ip_address': '192.168.0.9',
 'redis_address': '192.168.0.9:25256',
 'object_store_address': '/tmp/ray/session_2021-08-04_20-43-58_067921_27920/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-04_20-43-58_067921_27920/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-08-04_20-43-58_067921_27920',
 'metrics_export_port': 58424,
 'node_id': '161312dcd960744f3a910d623b4ef39a5f437209e3af02292fcceed3'}

In [5]:
call_anchors_from_hichip(
    bam_file,
    digestion_site,
    genome_fa,
    prefix,
    macs2_out_dir,
    exclude_interchro,
    local_range,
    macs2_path,
    macs2_qval,
    macs2_genome,
    mapq,
    nprocs,
)

2021-08-04 20:47:22,608: Parsing bam data to Hi-C pairs
2021-08-04 20:55:11,679: Removing duplications
2021-08-04 20:55:52,953: Processed 71414597 paired PETs. 24918351 paired PETs were kept after duplication removal
2021-08-04 20:55:52,956: Flattening to PETs to 100 bp reads
2021-08-04 20:57:32,846: 52848558 reads were dumped.
2021-08-04 20:57:32,851: Calling peaks by MACS2
2021-08-04 21:02:28,934: MACS2 called 145799 peaks at q < 0.01


'/Extension_HDD2/Hanbin/ES_Cell/E14/HiC3_HL/HL28_Smc1_MiDHiChIP_Test/HL28_Smc1_MiDHiChIP_out/Smc1_HiChIP_1kb_MACS2_results/Smc1_HiChIP_1kb_peaks.narrowPeak'

# Step 1: OBJECT call significant interactions

In [2]:
# skip this cell if starts from step 0
import ray, logging

ray.init(num_cpus=30)
logging.basicConfig(level=logging.INFO, format="%(asctime)s: %(message)s")

2021-08-04 23:03:31,424	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8266


In [3]:
from hichip_object.load_loop_data import process_peak_to_anchor_bins
from hichip_object.glm_loop_model import Loop_ZIP

Parameters

In [4]:
peak_file = "/Extension_HDD2/Hanbin/ES_Cell/E14/HiC3_HL/HL28_Smc1_MiDHiChIP_Test/HL28_Smc1_MiDHiChIP_out/Smc1_HiChIP_1kb_MACS2_results/Smc1_HiChIP_1kb_peaks.narrowPeak"
bin_size = 2500  # genome is bined and peaks are assigned to the genomic bins
chro_size = "/usr/local/bin/HiC-Pro_2.11.0-beta/annotation/chrom_mm9.sizes"
vp = "/Extension_HDD2/Hanbin/ES_Cell/E14/HiC3_HL/HL28_Smc1_MiDHiChIP_Test/HL28_Smc1_MiDHiChIP_out/hop_results/data.APASL.mppValidPairs"
inter_range = (5000, 2000000)

In [5]:
gb_anchors = process_peak_to_anchor_bins(
    peak_file=peak_file,
    chro_size=chro_size,
    resolution=bin_size,
)

Load validpair data and mapped to putative anchor pairs

In [6]:
loop_zip = Loop_ZIP(
    vp,
    gb_anchors,
    # inter_range = inter_range
)

2021-08-04 23:04:21,294: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-08-04 23:06:01,700: Loading loop PETs
2021-08-04 23:08:33,296: Counting anchor depth
2021-08-04 23:08:33,508: Joining data
2021-08-04 23:11:44,807: 83933 of anchors forms 7792801 putative mid-range loops
2021-08-04 23:11:44,870: 848138 observed loops that contains 2276923.0 PETs (avg = 2.6846138246370286)


Fit model and calculate p values

In [7]:
loop_zip.iteratively_fit_model(disp_glm_summary=False)

2021-08-04 23:13:10,491: ========Iter 1========
2021-08-04 23:13:10,492: Fitting data to ZeroInflatedPoisson model
2021-08-04 23:16:05,162: AIC: 6714920.058740447
2021-08-04 23:16:05,835: 51709 interactions were called at qval <= 0.1, count >= 2.6846138246370286
2021-08-04 23:16:05,836: ========Iter 2========
2021-08-04 23:16:06,373: 9976 high confident interactions were removed from fitting
2021-08-04 23:19:38,412: AIC: 5938309.422073574
2021-08-04 23:19:39,289: 62165 interactions were called at qval <= 0.1, count >= 2.6846138246370286
2021-08-04 23:19:39,290: Sum square changes of E: 82344.23883935733
2021-08-04 23:19:39,290: ========Iter 3========
2021-08-04 23:19:39,870: 11985 high confident interactions were removed from fitting
2021-08-04 23:23:13,352: AIC: 5865858.797640938
2021-08-04 23:23:14,020: 63655 interactions were called at qval <= 0.1, count >= 2.6846138246370286
2021-08-04 23:23:14,022: Sum square changes of E: 2932.11746670235
2021-08-04 23:23:14,022: ========Iter 4==

In [8]:
out = "loop_zip.self_1kb_peak.bedpe"
loop_zip.write_interaction_statistics(
    out,
)

# Test overlap with Micro-C loops

In [13]:
import pandas as pd
from hichip_object.loop_merge import (
    combine_two_loop_list,
    non_redundant_loops,
    significant_loops,
    mid_range_loops,
    drop_unplaced_chromosome_data,
    read_hicuups_loop,
)

In [11]:
microc_loop_file = "/Extension_HDD2/Hanbin/ES_Cell/Jm8.N4/Loops/WT_ALL_merge.allValidPairs_5_10kb_res_SCALE_hiccups/merged_loops.bedpe"
microc_loops = non_redundant_loops(
    mid_range_loops(
        drop_unplaced_chromosome_data(read_hicuups_loop(microc_loop_file))
    ).reset_index(drop=True),
    10000,
)

2021-08-04 23:36:08,971: 21030 reduces to 21030


In [16]:
out = "loop_zip.self_1kb_peak.bedpe"
hichip_hightier_loops = non_redundant_loops(
    significant_loops(
        drop_unplaced_chromosome_data(
            pd.read_csv(
                out,
                sep="\t",
                header=None,
                names=["chr1", "x1", "y1", "chr2", "x2", "y2", "counts", "qval"],
            )
        ),
        0.05,
        3,
    ).reset_index(drop=True),
    20000,
)
combine_two_loop_list(
    microc_loops, hichip_hightier_loops, 25000, ("Micro-C", "Smc1_MiDHiChIP")
)

2021-08-04 23:41:49,688: 51196 reduces to 33916
2021-08-04 23:41:49,801: 21030 records in "Micro-C" loop set; 51196 records in "Smc1_MiDHiChIP" loop set
2021-08-04 23:41:49,822: Building Graph
2021-08-04 23:41:55,519: Assigning Components
2021-08-04 23:42:29,543: Combined to 38797 components (merged loops combined graph of both loop set)
2021-08-04 23:42:29,548: Only "Micro-C": 8185 component are consist of 8489 "Micro-C" non-redundant loops
2021-08-04 23:42:29,551: Only "Smc1_MiDHiChIP": 20162 component are consist of 21056 "Smc1_MiDHiChIP" non-redundant loops
2021-08-04 23:42:29,562: 10450 overlaping components are equal to 12541 non-redundant loops for "Micro-C" or 12860 for "Smc1_MiDHiChIP"
